# Machine Project 6: EDGAR Web Logs

In [27]:
# Add imports used throughout the project here
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import geopandas as gpd
import zipfile
import pandas as pd
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Group Part (75%)

For this portion of the machine project, you may collaborate with your group members in any way (including looking at group members' code). You may also seek help from CS 320 course staff (peer mentors, TAs, and the instructor). You **may not** seek or receive help from other CS 320 students (outside of your group) or anybody else outside of the course.

## Part 1: `server_log.zip` analysis

In [2]:
# Use pandas to read in "server_log.zip as a csv file"
server_log = pd.read_csv("server_log.zip",compression = 'zip')
server_log.head()

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,104.197.32.ihd,2017-01-01,00:00:00,0.0,1111711.0,0001193125-12-324016,-index.htm,200.0,7627.0,1.0,0.0,0.0,10.0,0.0,NaN
1,208.77.214.jeh,2017-01-01,00:00:00,0.0,789019.0,0001193125-06-031505,.txt,200.0,46327.0,0.0,0.0,0.0,10.0,0.0,NaN
2,54.197.228.dbe,2017-01-01,00:00:00,0.0,800166.0,0001279569-16-003038,-index.htm,200.0,16414.0,1.0,0.0,0.0,10.0,0.0,NaN
3,108.39.205.jga,2017-01-01,00:00:01,0.0,354950.0,0000950123-09-011236,-index.htm,200.0,8718.0,1.0,0.0,0.0,10.0,0.0,NaN
4,52.45.218.ihf,2017-01-01,00:00:01,0.0,1313918.0,0001209191-06-031555,.txt,200.0,8911.0,0.0,0.0,0.0,10.0,0.0,NaN


### Q1: What's the total size in bytes of the files requested?

Look at the `size` column of the CSV in `server_log.zip`.  We want to include duplicates here; this gives us an estimate of the amount of network traffic handled by EDGAR (since this data is only a sample, the true value will be even larger). Answer with an integer. 

**Note:** If you use `numpy` make sure to cast the final answer to an `int`.

In [3]:
#Q1
int(server_log['size'].sum())

24801002666

### Q2: How many filings have been accessed by the 10 IPs with the most accesses?

Answer with a dictionary, with the (anonymized) IP as key and the number of requests seen in the logs as the values. Each row in the logs corresponds to one request. Note that the anonymized IP addresses are consistent between requests.

**Hint:** for this question and most of the others expecting dictionary output, it might be easiest to use Pandas operations to process the data into a `Series` and to use the `to_dict()` method. Consider using tools like `groupby`, `apply`, and aggregation methods like `size()`. In Q30-32 from [MP1](../mp1/README.md), there is an example of `apply`.


In [4]:
#Q2
server_log['ip'].value_counts()[:10].to_dict()

{'54.152.17.ccg': 12562,
 '183.195.251.hah': 6524,
 '52.45.218.ihf': 5562,
 '68.180.231.abf': 5493,
 '204.212.175.bch': 4708,
 '103.238.106.gif': 4428,
 '208.77.215.jeh': 3903,
 '208.77.214.jeh': 3806,
 '217.174.255.dgd': 3551,
 '82.13.163.caf': 3527}

### Q3: What fraction of the requests had errors?

Any request with a status code greater than or equal to 400 has an error. Answer with a floating point number. 

**Note:** If you use `numpy` make sure to cast the final answer to a `float`.

In [5]:
#Q3
float((server_log['code']>=400).sum()/len(server_log))

0.03466852724527611

### Q4: What is the second most frequently accessed file?

Answer with a string formatted like so: "cik/accession/extention" (these are the names of columns in "rows.csv").

In [6]:
#Q4
server_log_q4 = server_log
server_log_q4['cik'] = server_log['cik'].astype(int)
server_log_q4['file_name'] = server_log_q4[['cik','accession','extention']].astype(str).agg('/'.join, axis=1)
server_log_q4['file_name'].value_counts().index[1]

'1584509/0001584509-16-000514/armk-20160930_def.xml'

## Part 2: Creating `edgar_utils.py` module

This part is to be started during [Lab 9](../../labs/Lab9/README.md). Finish the `edgar_utils.py` module now if you didn't have enough time
during the scheduled lab.

## Part 3: Using `edgar_utils.py` module

### Q5: Which region accesses resources most heavily in `server_log.zip`?

Use your `lookup_region` function and answer with a string.

In [7]:
#Q5
import edgar_utils
server_log['region'] = server_log['ip'].apply(edgar_utils.lookup_region)
server_log['region'].value_counts().idxmax()

'United States of America'

### Q6: What fraction of IPs in each region are high-volume users?

Consider IPs which accessed more than 300 EDGAR resoures to be
high-volume. This might indicate machines running automated scraping
and analysis tasks.

Note that given the sampling done in the data, the true EDGAR usage of
these machines is likely to be even heavier.

Answer with a dictionary, where the keys are the regions and the
values are the fraction (in floating point form) of IPs from that
region classified as high-volume.

**Example:**

Say "United States of America" has four IPs:
* 1.1.1.1 appears 1200 times in the logs
* 2.2.2.2 appears 900 times in the logs
* 3.3.3.3 appears 5 times in the logs
* 4.4.4.4 appears 234 times in the logs

This means that 1/2 of the IPs in the US are high volume, so there should be an entry like this:

```
{
    "United States of America": 0.5,
    ...
}
```

**Note:** Some of the filings are listed as having a region of '-'. Please include this in your final
answer.

**Note:** If you use `numpy` make sure to cast dictionary entries to `float`.

In [8]:
#Q6
regions = set(server_log['region'])
frac_hcol = {region:0 for region in regions}
for region in frac_hcol:
    ips_of_this_region = server_log[server_log['region']==region]['ip']
    frac_hcol_of_this_region = float((ips_of_this_region.value_counts()>300).mean())
    frac_hcol[region] = frac_hcol_of_this_region
frac_hcol

{'Canada': 0.05357142857142857,
 'Luxembourg': 0.0,
 'North Macedonia': 0.0,
 'Finland': 0.0,
 'Mexico': 0.0,
 'Israel': 0.0,
 'Lebanon': 0.0,
 'Hungary': 0.0,
 'Honduras': 0.0,
 'Bangladesh': 0.0,
 'Turkey': 0.0,
 'Viet Nam': 0.0,
 'Sweden': 0.0,
 '-': 0.0,
 'Virgin Islands (British)': 0.0,
 'Ukraine': 0.0,
 'Cyprus': 0.0,
 'Greece': 0.0,
 'India': 0.08823529411764706,
 'Hong Kong': 0.037037037037037035,
 'Pakistan': 0.0,
 'Poland': 0.0,
 'United States of America': 0.04889228418640183,
 'Austria': 0.0,
 'Indonesia': 0.0,
 'Netherlands': 0.0,
 'Egypt': 0.0,
 'Slovenia': 0.0,
 'Syrian Arab Republic': 0.0,
 'Angola': 0.0,
 'Russian Federation': 0.0,
 'Portugal': 0.0,
 'Latvia': 0.0,
 'Estonia': 0.0,
 'Romania': 0.0,
 'Malaysia': 0.0,
 'Jordan': 0.0,
 'France': 0.0036900369003690036,
 'Spain': 0.0,
 'Slovakia': 0.0,
 'South Africa': 0.0,
 'Ireland': 0.625,
 'Sudan': 0.0,
 'Germany': 0.047619047619047616,
 'Algeria': 0.0,
 'Czechia': 0.3333333333333333,
 'Australia': 0.07692307692307693,


### Requirement: `filings` dictionary

Read every file ending with .htm or .html in `docs.zip`, and create a `Filing`
object based on that file. Then, save that `Filing` object to a dictionary as follows:
- **Key:** The filepath for this filing object (ex. `850693/0000850693-07-000159/-index.htm`)
- **Value:** The `Filing` object created from this filepath.

Creating this dictionary once now will save us from needing to loop over all values
in future questions.

In [14]:
# Create `filings` dictionary
filings = {}
with zipfile.ZipFile('docs.zip','r') as folder:
    for file in folder.namelist():
        if file.endswith(('html','htm')):
            with folder.open(file) as f:
                html_content = f.read().decode('utf-8') 
                filing_obj = edgar_utils.Filing(html_content)
                filings[file] = filing_obj
# filings

### Q7: What dates appear in the `886982/0000769993-16-001958/-index.htm` file of `docs.zip`?

Read the HTML from this file and use it to create a `Filing` object,
from which you can access the `.dates` attribute.

In [15]:
#Q7
filings['886982/0000769993-16-001958/-index.htm'].dates

['2016-12-30', '2016-12-30', '2016-12-20']

### Q8: What is the distribution of states for the filings in `docs.zip`?

Answer with a dict, like the following:

```
{'CA': 92,
 'NY': 83,
 'TX': 67,
 'None': 56,
 'MA': 30,
 'IL': 25,
 'PA': 25,
 'CO': 25,
 ...
}
```

The showing order of each key-value pair doesn't really matter. Please include `None` in the
dictionary.

**Hint:** We created the `filings` dictionary above, which means we don't have to
iterate through `docs.zip` here again!

In [44]:
#Q8
state_counts = {}
all_states = []
for item in filings:
    all_states.extend([''.join(re.findall(r'[A-Z]{2}',zipcode)) for zipcode in filings[item].state()])

for state in all_states:
    if state not in state_counts:
        state_counts[state]=1
    else:
        state_counts[state]+=1
state_counts

{'OX': 13,
 'CA': 707,
 'NJ': 69,
 'MS': 7,
 'MA': 91,
 'CO': 62,
 'NY': 351,
 'TX': 194,
 'FL': 55,
 'AL': 2,
 'IN': 15,
 'CT': 150,
 'UT': 14,
 'NV': 25,
 'MD': 25,
 'KS': 12,
 'MO': 11,
 'VA': 38,
 'OH': 22,
 'WA': 12,
 'SD': 3,
 'DE': 38,
 'GA': 106,
 'IL': 91,
 'NC': 34,
 'PA': 61,
 'TN': 7,
 'MN': 34,
 'NM': 2,
 'KY': 4,
 'OK': 15,
 'AR': 5,
 'LA': 12,
 'VT': 2,
 'MI': 25,
 'NE': 9,
 'IA': 16,
 'AZ': 15,
 'WI': 25,
 'OR': 6,
 'ID': 2,
 'WV': 4,
 'DC': 2,
 'ME': 2}

### Q9: What is the distribution for the ten most common addresses for the filings in `docs.zip`?

Answer in the same format as the previous question.

Expected output:
```
{'2000 AVENUE OF THE STARS, 12TH FLOOR\nLOS ANGELES CA 90067': 134,
 '2000 AVENUE OF THE STARS, 12TH FLOOR\nLOS ANGELES CA 90067\n3102014100': 113,
 '3 LANDMARK SQUARE\nSUITE 500\nSTAMFORD CT 06901\n2033564400': 60,
 'C/O KKR ASSET MANAGEMENT LLC\n555 CALIFORNIA STREET, 50TH FLOOR\nSAN FRANCISCO CA 94104': 36,
 'C/O ARES MANAGEMENT LLC\n2000 AVENUE OF THE STARS, 12TH FLOOR\nLOS ANGELES CA 90067': 35,
 '4740 AGAR DRIVE\nRICHMOND A1 V7B 1A3': 25,
 'CENTRALIS S.A., 8-10 AVENUE DE LA GARE\nLUXEMBOURG N4 L-1610': 25,
 'CENTRALIS S.A., 8-10 AVENUE DE LA GARE\nLUXEMBOURG N4 L-1610\n352-26-186-1': 25,
 '3 LANDMARK SQUARE\nSUITE 500\nSTAMFORD CT 06901': 24,
 '801 CHERRY STREET\nSUITE 2100\nFORT WORTH TX 76102': 22}
```

In [ ]:
#Q9

# Individual Part (25%)

For this portion of the machine project, you are only allowed to seek help from CS 320 course staff (peer mentors, TAs, and the instructor). You **may not** receive help from anyone else.

## Part 4: Combining logs with documents

### Q10: What is the distribution of requests across industries?

For each request in the logs that has a corresponding filing in
`docs.zip`, lookup the SIC (ignore rows in the logs which refer to
pages not in `docs.zip`).

Answer with a dictionary, where the keys are the SIC and the values
are the number of times the resources of that industry were accessed.

If you're curious, consider looking up the industry names for the top
couple categories:
https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list

Expected output:

```
{2834: 984,
 1389: 656,
 1311: 550,
 2836: 429,
 6022: 379,
 1000: 273,
 ...
 }
 ```

In [ ]:
#Q10


### Q11: How many requests were made in each hour?

Use `pd.to_datetime` (the `hour` attributes of the converted
timestamps may be useful) or string manipulation to process the `time`
column. Answer with a dictionary, where the keys are integers from 0
to 23 representing the hour of the day, and the values are the number
of requests made in that hour.

In [ ]:
#Q11

### Q12: What is the geographic overlap in interest between Australia, France, Indonesia, and Viet Nam?

Answer with a Digraph like the following:

<img src="img/digraph.png" width=400>

In addition to a node for each of these three countries, there should
be a node for each state having a filing accessed by somebody in one
of these countries.

An edge from a country to a state means somebody in that country
looked at least one filing for a company in that state.

**Important:** Make sure not to hardcode these values. It might be helpful to
define a list like `countries = ["Australia", "France", "Indonesia", "Viet Nam"]` and then loop over the filings for these countries only.

In [ ]:
#Q12
d = graphviz.Digraph()

# ADD CODE HERE

# IMPORTANT -- Do not remove -- 
with open("Q12.pkl", "wb") as f:
    pickle.dump(d.source, f)

d

### Q13: Geographic plotting of postal code

The `locations.geojson` contains the positions of some of the
addresses in the dataset.  Plot this over the background map in
"shapes/cb_2018_us_state_20m.shp"

Additional requirements:

* **Important:** Make sure to pass in `ax` as an argument when plotting: `.plot(ax=ax, ...)`
* Use the given lat/lon to crop the bounds:
* Use a Mercator projection, "epsg:2022"
* The color of each point should indicate the postal code. For example, the postal code of `245 SUMMER STREET\nBOSTON MA 02210` is `2210`. If it's in the form like `53705-1234`, only take `53705`. If it's neither 5 digits number nor 9 digits number, don't use the point.
* Only show the street with a postal code from 10000 to 60000
* Use the "viridis" colormap, with a colorbar
* The color of background is "lightgray"

The result should look similar to this:

<img src="img/geo.png" width="400px">

**Hint:** If you are getting issues with getting the slanted lines or issues with your graphs 
disappearing, make sure that you are applying the `.to_crs(...)` function to both the points
and the background before plotting.

In [ ]:
west = -90
east = -65
north = 50
south = 25

In [ ]:
#Q13

# ADD CODE HERE

# Create the plot
fig, ax = plt.subplots()

# PLOT HERE

# IMPORTANT -- Do not remove -- 
with open("Q13.pkl", "wb") as f:
    pickle.dump(fig, f)

plt.show()